In [1]:
from CTGAN_1_7 import *

In [2]:
import pandas as pd

train_data = pd.read_csv('../Data/Train Test/train_data.csv')[['Promoter Sequence', 'Normalized Expression']]
train_data = expand_sequences(train_data)
train_data.head()

,Base_0,Base_1,Base_2,Base_3,Base_4,Base_5,Base_6,Base_7,Base_8,Base_9,...,Base_141,Base_142,Base_143,Base_144,Base_145,Base_146,Base_147,Base_148,Base_149,Normalized Expression
0,T,T,T,T,C,T,A,T,C,T,...,-,-,-,-,-,-,-,-,-,0.185770
1,T,T,G,C,G,G,T,T,T,T,...,C,G,T,G,G,A,A,T,T,0.110559
2,A,G,C,C,G,C,T,T,T,T,...,C,T,G,C,T,A,A,C,C,0.124917
3,G,C,G,C,C,C,G,C,T,G,...,-,-,-,-,-,-,-,-,-,0.370848
4,G,C,G,C,C,C,G,C,T,G,...,-,-,-,-,-,-,-,-,-,0.108196


In [3]:
test_data = pd.read_csv('../Data/Train Test/test_data.csv')[['Promoter Sequence', 'Normalized Expression']]
test_data = expand_sequences(test_data)
test_data.head()

,Base_0,Base_1,Base_2,Base_3,Base_4,Base_5,Base_6,Base_7,Base_8,Base_9,...,Base_141,Base_142,Base_143,Base_144,Base_145,Base_146,Base_147,Base_148,Base_149,Normalized Expression
0,G,C,G,C,C,C,G,C,T,G,...,-,-,-,-,-,-,-,-,-,0.447876
1,G,T,T,A,A,C,A,A,A,C,...,G,A,A,G,C,T,C,C,T,0.098249
2,G,C,G,C,C,C,G,C,T,G,...,-,-,-,-,-,-,-,-,-,0.413318
3,G,C,G,C,C,C,G,C,T,G,...,-,-,-,-,-,-,-,-,-,0.451690
4,A,C,C,T,C,G,A,G,C,A,...,-,-,-,-,-,-,-,-,-,0.265830


In [4]:
from sdv.metadata import Metadata

# 1. auto-detect metadata based in your data
metadata = Metadata.detect_from_dataframes(
    data={
        'sequences': train_data,
    })

# 2. carefully inspect and update your metadata
metadata.visualize()
for column in train_data.columns[:-1]:
    metadata.update_column(
        column_name=column,
        sdtype='categorical',
        table_name='sequences'
    )
metadata.update_column(
    column_name='Normalized Expression',
    sdtype='numerical',
    table_name='sequences'
)

c:\Users\rsore\anaconda3\envs\TX_prediction\Lib\site-packages\sdv\metadata\visualization.py:118: RuntimeWarning: Graphviz does not seem to be installed on this system. For full metadata visualization capabilities, please make sure to have its binaries propertly installed: https://graphviz.gitlab.io/download/
  warnings.warn(warning_message, RuntimeWarning)


In [6]:
from sdv.tabular import TVAE

synthesizer = TVAE(metadata, epochs=100, verbose=True)
synthesizer.fit(train_data)

ModuleNotFoundError: No module named 'sdv.tabular'

In [ ]:
synthetic_data = synthesizer.sample(num_rows=500)
synthetic_data.head()

In [ ]:
from sdv.evaluation.single_table import run_diagnostic

diagnostic = run_diagnostic(
    real_data=train_data,
    synthetic_data=synthetic_data,
    metadata=metadata
)

In [ ]:
from sdv.evaluation.single_table import evaluate_quality

quality_report = evaluate_quality(
    test_data,
    synthetic_data,
    metadata
)